# Taller 1 NLP


Integrante: Erich Giussppe Soto Parada

Integrante: Cristobal

## Imports

In [77]:
import os
import re
import time
import unicodedata
import xml.etree.ElementTree as ET
from typing import List, Dict, Set

import nltk
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from langdetect import detect, detect_langs

import numpy as np

from gensim.corpora import Dictionary
from gensim import models, similarities
from gensim.utils import simple_preprocess

## [10p] Implemente las siguientes métricas de evaluación de IR usando python+numpy (debe usar numpy):

In [ ]:
def precision(relevance_arr: np.array) -> float:
    """function that evaluates the precicion of set of retrived documents

    Args:
        relevance_arr (np.array): An array containing binary values (1 and 0) indicating
        whether each document is relevant (1) or not relevant (0).
        The elements must be ordered by document index, from best to worst.

    Returns:
        float: precision of the retrival list
    """
    return np.mean(relevance_arr)


def precision_at_k(relevance_arr: np.array, k: int) -> float:
    """precision at k, precision obtained slicing ultil the k position

    Args:
        relevance_arr (np.array): An array containing binary values (1 and 0) indicating
        whether each document is relevant (1) or not relevant (0).
        The elements must be ordered by document index, from best to worst.
        k (int): k position where the slice is going to be made

    Returns:
        float: precision of the retrival list at k
    """
    return np.mean(relevance_arr[0:k])


def average_precision(relevance_arr: np.array):
    """Funcion to obtain the average pecision in diferent k,
    only k with document relevance are taking into account

    Args:
        relevance_arr (np.array): An array containing binary values (1 and 0) indicating
        whether each document is relevant (1) or not relevant (0).
        The elements must be ordered by document index, from best to worst.
        k (int): k position where the slice is going to be made

    Returns:
        _type_: average precision of the retrival list at k
    """

    precisiones = [
        precision_at_k(relevance_arr, k + 1)
        for k in range(len(relevance_arr))
        if relevance_arr[k] == 1
    ]
    if len(precisiones) == 0:
        return np.float64(0)
    else:
        return np.mean(precisiones)


def recall_at_k(relevance_arr: np.array, number_relevant_doc: int, k: int) -> float:
    """calculates the recall until the k position

    Args:
        relevance_arr (np.array): An array containing binary values (1 and 0) indicating
        whether each document is relevant (1) or not relevant (0).
        The elements must be ordered by document index, from best to worst.
        k (int): k position where the slice is going to be made
        number_relevant_doc (int): int

    Returns:
        float: _description_
    """
    return relevance_arr[0:k].sum() / number_relevant_doc


def mean_average_precision(querys_matrix: np.ndarray) -> float:
    """mean of the average precision across queries

    Args:
        querys_matrix (np.ndarray): A list of lists containg each list being
        an array containing binary values (1 and 0) indicating
        whether each document is relevant (1) or not relevant (0).
        The elements must be ordered by document index, from best to worst.
        k (int): k position where the slice is going to be made

    Returns:
        float: mean average precision for the imput queries
    """

    # scores = [average_precision(querys_matrix) for row in querys_matrix]
    querys_matrix = np.array(querys_matrix, dtype=object)
    arr_result = np.zeros(len(querys_matrix))
    for i, row in enumerate(querys_matrix):
        arr_result[i] = average_precision(row)  # cada row puede tener tamaño distinto
    return np.mean(arr_result)  # , arr_result


def average_recall(relevance_arr: np.array, number_relevant_doc: int) -> float:
    """average recall of the k evaluated positions,

    Args:
        relevance_arr (np.array): An array containing binary values (1 and 0) indicating
        whether each document is relevant (1) or not relevant (0).
        The elements must be ordered by document index, from best to worst.
        number_relevant_doc (int): Number of relevant documents

    Returns:
        float: average recall for the relevance_arr
    """

    recalls = [
        recall_at_k(relevance_arr, number_relevant_doc, k + 1)
        for k in range(len(relevance_arr))  # se debe solo tener en cuenta esto ?
        if relevance_arr[k] == 1
    ]
    if len(recalls) == 0:
        return np.float64(0)
    else:
        return np.mean(recalls)


def mean_average_recall(
    querys_matrix: list[np.array], relevant_counts: list[int]
) -> float:
    """calculates que mean average recall for the querys matrix and the list of relevant counts

    Args:
        querys_matrix (list[np.array]): A list of lists containg each list being
        an array containing binary values (1 and 0) indicating
        whether each document is relevant (1) or not relevant (0).
        The elements must be ordered by document index, from best to worst.
        k (int): k position where the slice is going to be made
        relevant_counts (list[int]): Array with the number of relevant documents for each query

    Returns:
        float: mean average recall of the querys
    """

    arr_result = np.zeros(len(querys_matrix))
    for i, row in enumerate(querys_matrix):
        arr_result[i] = average_recall(row, relevant_counts[i])
    return np.mean(arr_result)  # ,arr_result


def dct_at_one_k(relevance_arr: np.array, k: int) -> float:
    """DCT of the relevance array

    Args:
        relevance_arr (np.array): An array containing whole values indicating
        whether each document is relevant (positive number) or not relevant (0).
        k (int): k position where the slice is going to be made

    Returns:
        float: float containing the dct at k of the query information retrieval
    """

    gain = np.sum(relevance_arr)
    discount_factor = np.log2(max(k, 2))
    return gain / discount_factor


def dcg_at_k(relevance_arr: np.array, k: int) -> float:
    """DCT average for each k in the relevance array or result of the query

    Args:
        relevance_arr (np.array): An array containing whole values indicating
        whether each document is relevant (positive number) or not relevant (0).
        k (int): k position till the average is going to be made

    Returns:
        float: _description_
    """
    expected2 = [
        value / np.log2(max(2, i + 1)) for i, value in enumerate(relevance_arr[:k])
    ]
    expected2 = np.sum(expected2)

    return expected2


def ndcg_at_k(relevance_arr: np.array, k: int) -> float:
    """Normalized DCG for each k in the relevance array or result of the query

    Args:
        relevance_arr (np.array): _description_
        k (int): k position till the average is going to be made

    Returns:
        float: ndcg float
    """
    dcg = dcg_at_k(relevance_arr, k)
    ideal_dcg = dcg_at_k(sorted(relevance_arr, reverse=True), k)
    if ideal_dcg == 0:
        return 0.0
    return dcg / ideal_dcg


# TODO faltan las pruebas

In [79]:
# Pruebas


def test_precision():
    assert precision(np.array([0, 0, 0, 1])) == np.float64(0.25)
    assert precision(np.array([1, 1, 1, 1])) == np.float64(1.0)
    assert precision(np.array([0, 0, 0, 0])) == np.float64(0.0)
    assert precision(np.array([1, 0, 1, 0])) == np.float64(0.5)


def test_precision_at_k():
    assert precision_at_k(np.array([0, 0, 0, 1]), 2) == np.float64(0.0)
    assert precision_at_k(np.array([1, 1, 1, 1]), 2) == np.float64(1.0)
    assert precision_at_k(np.array([1, 0, 1, 0]), 2) == np.float64(0.5)
    assert precision_at_k(np.array([1, 0, 1, 0]), 3) == np.float64(2 / 3)


def test_recall_at_k():
    arr = np.array([0, 0, 0, 1])
    assert recall_at_k(arr, number_relevant_doc=1, k=2) == 0.0
    assert recall_at_k(arr, number_relevant_doc=1, k=4) == 1.0
    assert recall_at_k(arr, number_relevant_doc=4, k=1) == 0

    arr = np.array([1, 1, 1, 1])
    assert recall_at_k(arr, number_relevant_doc=4, k=2) == 0.5
    assert recall_at_k(arr, number_relevant_doc=4, k=4) == 1.0

    arr = np.array([1, 0, 1, 0])
    assert recall_at_k(arr, number_relevant_doc=2, k=2) == 0.5
    assert recall_at_k(arr, number_relevant_doc=2, k=3) == 1.0


def test_average_precision():

    relevance_query_2 = [0, 1, 0, 1, 1, 1, 1]
    result = average_precision(relevance_query_2)
    assert np.isclose(result, 0.5961904, atol=1e-6)

    relevance_all = [1, 1, 1, 1]
    assert average_precision(relevance_all) == 1.0

    relevance_none = [0, 0, 0, 0]
    assert average_precision(relevance_none) == 0.0

    relevance_last = [0, 0, 0, 1]
    assert average_precision(relevance_last) == 0.25


def test_mean_average_precision():
    querys_matrix = np.array(
        [[0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 1]], dtype=object
    )

    result = mean_average_precision(querys_matrix)
    print(result, np.mean(result))
    expected = np.mean([0.5961904, 1.0, 0.0, 0.25])
    assert np.isclose(result, expected, atol=1e-6)
    querys_matrix = [
        [0, 1, 0, 1, 1, 1, 1],
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 1, 0, 1],
    ]
    result = mean_average_precision(querys_matrix)
    assert np.isclose(result, np.float64(0.471964285714), atol=1e-6)


def test_dcg_at_k():
    rels1 = np.array([1, 1, 1, 1])
    expected1 = 1 / 1 + 1 / 1 + 1 / np.log2(3) + 1 / np.log2(4)
    assert np.isclose(dcg_at_k(rels1, 4), expected1, atol=1e-6)

    rels2 = np.array([1, 0, 0, 0])
    expected1 = 1 / np.log2(2) + 0 / np.log2(2) + 0 / np.log2(3) + 0 / np.log2(4)
    assert np.isclose(dcg_at_k(rels2, 4), 1.0, atol=1e-6)

    rels3 = np.array([3, 2, 3, 0, 1])
    expected3 = 3 / np.log2(2) + 2 / np.log2(2) + 3 / np.log2(3) + 0 + 1 / np.log2(5)
    assert np.isclose(dcg_at_k(rels3, 5), expected3, atol=1e-6)


def test_ndcg_at_k():
    rels1 = np.array([1, 1, 1, 1])
    assert np.isclose(ndcg_at_k(rels1, 4), 1.0, atol=1e-6)

    rels2 = np.array([1, 0, 0, 0])
    assert np.isclose(ndcg_at_k(rels2, 4), 1.0, atol=1e-6)

    rels3 = np.array([3, 2, 3, 0, 1])
    dcg = dcg_at_k(rels3, 5)
    idcg = dcg_at_k(np.sort(rels3)[::-1], 5)
    expected_ndcg = dcg / idcg
    assert np.isclose(ndcg_at_k(rels3, 5), expected_ndcg, atol=1e-6)


test_precision()
print("Se paso la prueba de precision")
test_precision_at_k()
print("Se paso la prueba de precision at k")
test_recall_at_k()
print("Se paso la prueba de recall at k")
test_average_precision()
print("se paso la prueba de average precision")
test_mean_average_precision()
print("Se paso el test de mean average precision")
test_dcg_at_k()
print("Se paso la prueba de dcg_at_k")
test_ndcg_at_k()
print("Se paso la prueba de n_dcg_at_k")

Se paso la prueba de precision
Se paso la prueba de precision at k
Se paso la prueba de recall at k
se paso la prueba de average precision
0.46154761904761904 0.46154761904761904
Se paso el test de mean average precision
Se paso la prueba de dcg_at_k
Se paso la prueba de n_dcg_at_k


## [25p] Búsqueda binaria usando índice invertido (BSII)

### Procesamiento de texto

In [80]:
list_docs_files = sorted(os.listdir("./data/docs-raw-texts/"))
list_docs_files[:4]

['wes2015.d001.naf',
 'wes2015.d002.naf',
 'wes2015.d003.naf',
 'wes2015.d004.naf']

In [ ]:
def get_content_and_title(document: str) -> str:
    """function to get the title and the content and concatenate them

    Args:
        document (str): name of the document inside ./data/docs-raw-texts/

    Returns:
        str: document content and title concatenated
    """
    tree = ET.parse(f"./data/docs-raw-texts/{document}")
    root = tree.getroot()
    file_desc = root.find(".//fileDesc")
    element_raw = root.find("raw")
    if file_desc is not None:
        title = file_desc.attrib.get("title")
    text = f"{title}\n {element_raw.text}"
    return text


def get_content(document: str) -> str:
    """get only the content (used for the queries)

    Args:
        document (str): name of the document inside ./data/queries-raw-texts/

    Returns:
        str: string contaning the content of the query
    """
    tree = ET.parse(f"./data/queries-raw-texts/{document}")
    root = tree.getroot()
    element_raw = root.find("raw")
    return element_raw.text


# def process_text_stop_words(text):
#     text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
#     # Normalizacion
#     text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
#     text_no_stop_words = set(nltk.corpus.stopwords.words("english"))
#     # Tokenizacion
#     word_tok_nltk_en = nltk.word_tokenize(text)
#     word_tok_nltk_en_sw = [
#         token for token in word_tok_nltk_en if token not in text_no_stop_words
#     ]
#     # Stemming
#     snow_stemmer = nltk.stem.SnowballStemmer("english")
#     nltk_stemedList_en = [snow_stemmer.stem(word) for word in word_tok_nltk_en_sw]
#     return nltk_stemedList_en


STOP_WORDS_EN = nltk.corpus.stopwords.words("english")
STEMMER_EN = nltk.stem.PorterStemmer()

STOP_WORDS_DU = nltk.corpus.stopwords.words("german")
STEMMER_DU = nltk.stem.SnowballStemmer("german")


def process_text_stop_words(text: str) -> list[str]:
    """Process the keywords to make it tokens

    Args:
        text (str): raw text

    Returns:
        list[str]: tokens made from the raw text
    """
    # Normalización Unicode -> ASCII
    idioma = detect(text)
    # text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
    text = text.lower()
    stopwords = None
    tokenizer = None
    # Used because there is a document in deutsche
    if idioma == "en":
        stopwords = STOP_WORDS_EN  # MAP: 0.7757273205819124 MAR: 0.40534941249226963
        tokenizer = STEMMER_EN
    else:
        stopwords = STOP_WORDS_DU
        tokenizer = STEMMER_DU
    # Tokenización
    tokens = nltk.word_tokenize(text)

    # Filtrado
    clean_tokens = []
    for tok in tokens:
        tok = re.sub(r"[^A-Za-z0-9]+", "", tok)
        if (
            tok
            and tok not in stopwords
            and (tok.isalpha())
            and 2 <= len(tokenizer.stem(tok)) <= 20
        ):
            if tokenizer.stem(tok):
                clean_tokens.append(tokenizer.stem(tok))
    return clean_tokens


# def process_text_stop_words(text: str) -> list[str]:
#     """Normalize, tokenize, remove stopwords and apply stemming.

#     Args:
#         text (str): Raw text input.

#     Returns:
#         list[str]: List of processed tokens.
#     """
#     # Detectar idioma
#     idioma = detect(text)
#     if idioma == "en":
#         stop_words = STOP_WORDS_EN
#         stemmer = STEMMER_EN
#     else:
#         stop_words = STOP_WORDS_DU
#         stemmer = STEMMER_DU

#     # Normalización Unicode -> ASCII
#     text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
#     text = text.lower().strip()

#     # Regex limpieza (mantiene letras, números y espacios)
#     text = re.sub(r"[^a-zA-Z\s]", " ", text, flags=re.I | re.A | re.MULTILINE)

#     # Tokenización
#     tokens = nltk.word_tokenize(text)

#     # Stopwords + Stemming
#     filtered_tokens = [
#         stemmer.stem(token) for token in tokens
#         if token not in stop_words and token.isalpha()
#     ]

#     return filtered_tokens
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")
stop_words = nltk.corpus.stopwords.words("english")

wptk = nltk.WordPunctTokenizer()
ps = PorterStemmer()
lm = WordNetLemmatizer()


def process_text_stop_words(text: str) -> list[str]:
    """Process the keywords to make it tokens

    Args:
        text (str): raw text

    Returns:
        list[str]: tokens made from the raw text
    """
    doc = re.sub(r"[^a-zA-Z1-9\s]", " ", text, flags=re.I | re.A | re.MULTILINE)
    doc = doc.lower()
    doc = doc.strip()
    tokens = wptk.tokenize(doc)
    filtered_tokens = [ps.stem(token) for token in tokens if token not in stop_words]
    # filtered_tokens = [lm.lemmatize(token) for token in tokens if token not in stop_words]
    return filtered_tokens

[nltk_data] Downloading package stopwords to /home/erich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/erich/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [107]:
raw_text_example = """A large disadvantage of the BERT network
structure is that no independent sentence embeddings are computed, which makes it difficult to derive sentence embeddings from BERT. To bypass
this limitations, researchers passed single sentences through BERT and then derive a fixed sized
vector by either averaging the outputs"""
process_text_stop_words(raw_text_example)

['larg',
 'disadvantag',
 'bert',
 'network',
 'structur',
 'independ',
 'sentenc',
 'embed',
 'comput',
 'make',
 'difficult',
 'deriv',
 'sentenc',
 'embed',
 'bert',
 'bypass',
 'limit',
 'research',
 'pass',
 'singl',
 'sentenc',
 'bert',
 'deriv',
 'fix',
 'size',
 'vector',
 'either',
 'averag',
 'output']

### Creacion de indice invertido

Para la creacion del indice invertido se tienen dos funciones, la primera es la implementacion a partir de las diapositivas, primero, recorre cada documento de la lista, obtiene su contenido, y genera los tokens filtrando palabras vacías; a continuación, asigna a cada token el identificador del documento y los guarda en una lista, mientras mantiene un registro del vocabulario y los IDs de documentos; luego, ordena esta lista por palabra y documento; después, recorre la lista ordenada contando la frecuencia de cada término por documento, actualizando el índice invertido con la frecuencia de documento y las frecuencias por documento; finalmente, devuelve el índice invertido junto con el conjunto de IDs de documentos y el vocabulario completo.

In [ ]:
def inverted_index_f(document_list: list[str]) -> dict[object]:
    """function that returns an inverted index from a list of documets
    where each element has the shape [word, df, [[doc_id, tf], ...]]

    Args:
        document_list (list[str]): _description_

    Returns:
        dict[object]: _description_
    """
    words = []
    index: Dict[str, list] = {}
    doc_ids: Set[int] = set()
    vocab: Set[str] = set()
    # Paso 1 Generar los tokens con los docs
    for document in document_list:
        text = get_content_and_title(document)
        tokens = process_text_stop_words(text)
        doc_index = int(document.split(".")[1].strip("d"))
        doc_ids.add(doc_index)
        vocab.update(tokens)
        for token in tokens:
            words.append((token, doc_index))
    # Paso 2 ordenar las listas
    words.sort(key=lambda x: (x[0], x[1]))
    # Paso 3 contar cuantas veces esta la pa
    current_token = None
    current_doc = None
    tf = 0
    for term, doc in words:
        if term != current_token or doc != current_doc:
            if current_token is not None:
                if current_token not in index:
                    index[current_token] = [current_token, 1, [[current_doc, tf]]]
                else:
                    index[current_token][1] += 1
                    index[current_token][2].append([current_doc, tf])
            current_token = term
            current_doc = doc
            tf = 1
        else:
            tf += 1

    if current_token is not None:
        if current_token not in index:
            index[current_token] = [current_token, 1, [[current_doc, tf]]]
        else:
            index[current_token][1] += 1
            index[current_token][2].append([current_doc, tf])

    return index, doc_ids, vocab

La segunda opción será la que se utilice, no por ser más eficiente, sino porque todas las tareas de esta y de los puntos siguientes ya están implementadas en la clase. La idea es que, si alguien se tiene curiosidad, lea el docstring incluido en el documento; en pocas palabras, se trata de una tabla hash con funciones de índice invertido. Aunque es considerablemente menos eficiente que usar diccionarios, se asumió que el objetivo era implementar la estructura desde cero.

In [ ]:
class InvertedIndex:
    """
    Implementación de un Índice Invertido basado en una tabla hash.
    (haberlo hecho de la forma de las diapositivas es mas optimo y la implementacion esta arriba)
    Esta clase proporciona una estructura para almacenar y recuperar
    términos (palabras) junto con las listas de postings que indican en qué
    documentos aparecen y con qué frecuencia.

    El índice se implementa sobre una tabla hash con resolución de colisiones
    mediante chaining. Además, incluye soporte para:

    - Inserción de términos y actualización de frecuencias (TF, DF).
    - Consultas booleanas binarias (AND / NOT).
    - Consultas de texto libre.
    - Cálculo de representaciones vectoriales TF-IDF para documentos y queries.
    - Similitud coseno entre documentos y queries.
    - Construcción de una matriz de documentos (TF-IDF normalizada).
    - Expansión automática de la tabla hash (rehashing) cuando se supera
    un factor de carga.

    Atributos
    ---------
    hash_map : numpy.ndarray
        Tabla hash que almacena las entradas en forma de listas:
        [word, df, [[doc_index, tf], ...]].
    length : int
        Longitud de la tabla hash (número de posiciones disponibles).
    num_elements : int
        Número de términos actualmente almacenados en la tabla.
    word_list : list[str]
        Vocabulario actual (lista de todas las palabras insertadas).
    docs : list
        Lista de documentos asociados al índice. Usada principalmente
        para el cálculo de TF-IDF.

    Ejemplo
    -------
    >>> idx = InvertedIndex(length=10_000)
    >>> idx.insert("python", 0)
    True
    >>> idx.insert("python", 0)  # inserción repetida en el mismo doc -> aumenta TF
    True
    >>> idx.insert("ai", 1)
    True

    >>> idx.get("python")
    ['python', 1, [[0, 2]]]

    >>> idx.free_text_query("python ai")
    [0, 1]

    >>> vec_query = idx.create_tf_idf(doc_id=None, token_list=["python"], is_document=False)
    >>> resultados = idx.cosine_search(vec_query)
    [(0, 0.9123), (1, 0.3010)]
    """

    def __init__(self, length: int = 100_000):
        """Initialization of the inverted index

        Args:
            length (int, optional): lenght of the generated index. Defaults to 100_000.
        """
        self.hash_map = np.full(length, None, dtype=object)
        self.length = length
        self.num_elements = 0
        self.word_list = []
        self.docs = []

    def _get_index_hash(self, word: str) -> int:
        """Get the hash of the function

        Args:
            word (str): string with a hash will be calculated on

        Returns:
            int: position that the hash determined
        """
        word_index = hash(word) % self.length
        return word_index

    def insert(
        self,
        word: str,
        doc_index: int,
    ):
        """Inserta una palabra en la tabla hash y actualiza su lista de postings.

        Este método implementa la inserción de términos en un índice invertido
        usando una tabla hash como estructura base. Cada entrada de la tabla
        mantiene la siguiente estructura:

            [word, df, [[doc_index, tf], [doc_index, tf], ...]]

        Donde:
            - word (str): término a indexar.
            - df (int): document frequency, número de documentos distintos
            donde aparece la palabra.
            - posting_list (list): lista de pares [doc_index, tf], donde:
                - doc_index (int): identificador del documento.
                - tf (int): frecuencia de la palabra dentro de ese documento.

        Flujo general:
            1. Si la palabra no existe en el índice:
            - Se crea una nueva entrada con df=1 y una posting list inicial
                con [[doc_index, 1]].
            2. Si la palabra ya existe:
            - Se busca si el documento ya está en la posting list.
                - Si sí: se incrementa el tf de ese documento.
                - Si no: se incrementa df y se añade un nuevo par [doc_index, 1].

        Parámetros
        ----------
        word : str
            Palabra a insertar en el índice.
        doc_index : int
            Identificador del documento donde aparece la palabra.

        Retorna
        -------
        bool
            True si la operación de inserción o actualización se realizó con éxito,
            False en caso contrario.

        Ejemplo
        -------
        >>> index.insert("python", 3)
        True
        # Si "python" no estaba en el índice, se crea con df=1 y posting_list=[[3,1]].
        # Si ya estaba y doc_index=3 existe, se incrementa tf.
        # Si ya estaba y doc_index=3 no existe, se añade a la posting list.
        """
        if word not in self.word_list:
            self.word_list.append(word)
        self._check_load()
        # Calculate index
        word_index = self._get_index_hash(word)
        # Insert

        element_in_list = self.hash_map[word_index]

        if element_in_list is None:
            # posting_list ahora guarda pares [doc_index, tf]
            self.hash_map[word_index] = [[word, 1, [[doc_index, 1]]]]
            self.num_elements += 1
            return True
        else:
            i = 0
            while True:
                if i < len(element_in_list):
                    if element_in_list[i][0] == word:
                        # Buscar si el doc ya existe en la posting list
                        posting_list = element_in_list[i][2]
                        j = 0
                        while j < len(posting_list):
                            if posting_list[j][0] == doc_index:
                                # Incrementar TF para (word, doc)
                                posting_list[j][1] += 1
                                return True
                            j += 1
                        # Nuevo doc para la palabra: df += 1, tf inicia en 1
                        element_in_list[i][1] += 1
                        posting_list.append([doc_index, 1])
                        return True
                else:
                    element_in_list.append([word, 1, [[doc_index, 1]]])
                    self.num_elements += 1
                    return True
                i += 1

    def _check_load(self):
        """
        Checks whether the load factor is greater than or equal to 0.7.
        If so, triggers a rehash to expand the hash table and redistribute elements.
        """
        load_factor = self.num_elements / self.length
        if load_factor > 0.7:
            self._rehash()

    def _rehash(self):
        print("RE HASHING")
        """
        Performs a rehash of the inverted index.

        This operation doubles the size of the hash table and reinserts each word
        into the new structure.
        """
        old_hash = self.hash_map
        new_list_hash = InvertedIndex(length=self.length * 2)
        new_length = self.length * 2
        for bucket in old_hash:
            if bucket is None:
                continue
            for word, doc_freq, posting_list in bucket:
                # Reinsertar respetando los TF
                for doc_index, tf in posting_list:
                    for _ in range(tf):
                        new_list_hash.insert(word, doc_index)
        self.hash_map = new_list_hash.hash_map
        self.length = new_length
        self.num_elements = new_list_hash.num_elements

    def get(self, word: str) -> list[object]:
        """get the word position gavev by the hash, to get the relevant atributtes concerning the word

        Args:
            word (str): word that is being searched

        Returns:
            list: [word, df, [[doc_index, tf], [doc_index, tf], ...]]
        """
        index_hash = self._get_index_hash(word)
        bucket = self.hash_map[index_hash]

        if bucket is None:
            return None
        for element in bucket:
            if element[0] == word:
                return element
        return False

    @staticmethod
    def algoritmo_de_mezcla(list1: list, list2: list, is_and=True) -> list[object]:
        """
        Implements the merging algorithm for two sorted lists.

        This method assumes both input lists are sorted. Depending on the value
        of `is_and`, it performs either:

        - **Intersection (AND)**: Returns the common elements present in both lists.
        - **Difference (NOT)**: Returns the elements from `list1` that are not
        present in `list2`.

        Parameters
        ----------
        list1 : list
            The left input list.
        list2 : list
            The right input list.
        is_and : bool, optional, default=True
            If True, computes the intersection (AND) of the two lists.
            If False, computes the difference (list1 NOT list2).

        Returns
        -------
        list of object
            A list containing the result of the merge operation.

        Examples
        --------
        >>> algoritmo_de_mezcla([1, 2, 3, 4], [2, 4, 6], is_and=True)
        [2, 4]

        >>> algoritmo_de_mezcla([1, 2, 3, 4], [2, 4, 6], is_and=False)
        [1, 3]
        """

        i, j = 0, 0
        lista_final = []
        if is_and:
            while i < len(list1) and j < len(list2):
                if list1[i] > list2[j]:
                    j += 1
                elif list1[i] == list2[j]:
                    lista_final.append(list1[i])
                    j += 1
                    i += 1
                else:
                    i += 1
        else:
            while i < len(list1) and j < len(list2):
                if list1[i] > list2[j]:
                    j += 1
                elif list1[i] == list2[j]:
                    j += 1
                    i += 1
                else:
                    lista_final.append(list1[i])
                    i += 1
        return lista_final

    def free_token_query_binary(
        self, tokens_and_operators: list
    ) -> (
        list
    ):  # No es free text porque asumimos que esta normalizado y con los and etc de pormedio
        """
        Ejecuta una consulta booleana binaria sobre el índice invertido,
        asumiendo que la lista de entrada contiene términos normalizados y
        operadores lógicos (por ahora solo AND / NOT).

        La consulta se evalúa secuencialmente siguiendo el patrón:
        `term OP term [OP term]*`, donde `OP` puede ser "AND" o "NOT".

        Args:
            tokens_and_operators (list):
                Lista que alterna términos y operadores.
                Ejemplo: ["apple", "AND", "banana", "NOT", "cherry"]

        Raises:
            ValueError:
                Si la consulta no cumple el patrón esperado
                (menos de 3 elementos o cantidad par de tokens/operadores).

        Returns:
            list:
                Lista de identificadores de documentos que cumplen
                con la consulta booleana.
                Devuelve una lista vacía si no se encuentran coincidencias.
        """
        n = len(tokens_and_operators)
        if n < 3 or n % 2 == 0:
            raise ValueError(
                f"Entrada inválida: se espera patrón 'term OP term [OP term]*' Recibido n={n}."
            )

        w1, op, w2 = tokens_and_operators[0:3]
        operador = True if op == "AND" else False
        # TODO: Arreglar la transformacion del operador

        elem_1 = self.get(w1)
        elem_2 = self.get(w2)

        if (elem_1 is None or elem_1 is False) or (
            (elem_2 is None or elem_2 is False) and operador
        ):
            return []

        # Extraer solo los doc_index desde las posting lists con TF
        list_1 = [doc for doc, tf in elem_1[2]]
        list_2 = [doc for doc, tf in elem_2[2]]

        if len(list_1) == 0 or (len(list_2) == 0 and operador):
            return []

        accumulative = self.algoritmo_de_mezcla(list_1, list_2, operador)

        for i in range(3, len(tokens_and_operators), 2):

            op, w2 = tokens_and_operators[i : i + 2]

            operador = True if op == "AND" else False

            # TODO: Arreglar la transformacion del operador

            elem_2 = self.get(w2)
            if (elem_2 is None or elem_2 is False) and operador:
                return []

            list_2 = (
                []
                if (elem_2 is None or elem_2 is False)
                else [doc for doc, tf in elem_2[2]]
            )

            accumulative = self.algoritmo_de_mezcla(accumulative, list_2, operador)

            if len(accumulative) == 0:
                return []

        return accumulative

        # def free_token_query_cosine_similarity(self):
        #     """crea
        #     """
        #     for document in self.docs:
        #         self.create_tf_idf(document)

    def free_text_query(self, text: str, personalized_list=False):
        """
        Ejecuta una consulta de texto libre sobre el índice invertido.

        El texto ingresado se transforma en una consulta booleana
        básica reemplazando los espacios por el operador "AND".
        Posteriormente, se tokeniza y se delega la resolución a
        `free_token_query_binary`.

        Args:
            text (str):
                Consulta de entrada escrita en lenguaje natural o
                en forma de texto libre.
                Ejemplo: "apple banana cherry"
                Se convierte internamente en:
                ["apple", "AND", "banana", "AND", "cherry"]

            binary (bool, optional):
                Parámetro reservado para futuras extensiones.
                Actualmente no se utiliza. Por defecto es False.

        Returns:
            list:
                Lista de identificadores de documentos que cumplen
                con la consulta generada.
                Si no se encuentran coincidencias, devuelve una lista vacía.
        """
        if not personalized_list:
            text = text.replace(" ", " AND ")
        lista_query = text.split(" ")
        return self.free_token_query_binary(lista_query)

    def create_tf_idf(self, doc_id=None, token_list=None, is_document=True):
        """free_text_query
        Creates a TF-IDF vector for a given document or a query.

        This method computes the Term Frequency–Inverse Document Frequency (TF-IDF)
        representation over the vocabulary (`self.word_list`). The resulting vector
        has length equal to the number of terms in the index.

        Depending on `is_document`, the function behaves in two modes:

        - **Document mode (is_document=True)**:
        Builds the TF-IDF vector for a specific document already stored in the index.
        It uses the stored term frequencies from the postings list.

        - **Query mode (is_document=False)**:
        Builds the TF-IDF vector for an arbitrary token list (e.g., a query).
        In this case, term frequencies are computed directly from the `token_list`.

        Parameters
        ----------
        doc_id : int
            Identifier of the document to process. Used only if `is_document=True`.
        token_list : list of str, optional
            List of tokens to process (e.g., a query). Required if `is_document=False`.
        is_document : bool, default=True
            If True, computes the TF-IDF vector for the document `doc_id`.
            If False, computes the TF-IDF vector for the given `token_list`.

        Returns
        -------
        numpy.ndarray
            A 1D NumPy array of length `len(self.word_list)` containing
            the TF-IDF weights.

        Notes
        -----
        - TF is computed as `log10(tf + 1)`.
        - IDF is computed as `log10(N / df)`, where:
            * `N` is the total number of documents in the index.
            * `df` is the document frequency of the word.
        - Terms with `idf == 0` or `tf == 0` are logged for debugging.

        Examples
        --------
        >>> index.create_tf_idf(doc_id=5)
        array([0.0, 0.3010, 0.0, 0.4771, ...])

        >>> index.create_tf_idf(doc_id=None, token_list=["python", "ai"], is_document=False)
        array([0.1761, 0.0, 0.3010, ...])
        """
        vect = np.zeros(len(self.word_list))
        for idx, word in enumerate(self.word_list):
            bucket = self.get(word)
            bucket_docs = bucket[2]
            df = bucket[1]
            N = len(self.docs)
            idf = np.log10(N / df)
            if idf == 0:
                print(word)
            if idf != 0:
                if is_document:
                    for doc in bucket_docs:
                        doc_index = doc[0]
                        if doc_index == doc_id:
                            tf = doc[1]
                            if tf == 0:
                                print(word)

                            tf_idf = np.log10(tf + 1) * idf
                            vect[idx] = tf_idf
                            continue
                else:
                    tf = 0
                    for token in token_list:
                        if token == word:
                            tf += 1
                    tf_idf = np.log10(tf + 1) * idf
                    vect[idx] = tf_idf
            else:
                print(word)
        # print(count_word_enter)
        return vect

    def cosine_similarity(self, vect1: list, vect2: list) -> float:
        """Calculates the cosine similaritie

        Args:
            vect1 (list): vector 1
            vect2 (list): vector 2

        Returns:
            float: cosine similarity between the vect1 and vect2
        """
        dot_product = np.dot(vect1, vect2)
        norma_l2_v1 = np.linalg.norm(vect1)
        norma_l2_v2 = np.linalg.norm(vect2)
        if norma_l2_v1 == 0.0 or norma_l2_v2 == 0.0:
            return 0.0
        cosine_similarity = dot_product / (norma_l2_v1 * norma_l2_v2)
        return cosine_similarity

    def _build_document_matrix(self, normalize=True, force_recompute=False):
        """
        Construye (o recupera en caché) la matriz de representación de documentos
        a partir del índice invertido, utilizando TF-IDF como vector de características.

        La matriz resultante tiene dimensiones (N, V), donde:
        - N = número de documentos en la colección.
        - V = tamaño del vocabulario (`self.word_list`).

        Si la matriz ya fue calculada previamente, se devuelve desde
        caché a menos que `force_recompute=True`.

        Args:
            normalize (bool, optional):
                Si es True, devuelve una versión normalizada de la matriz
                (filas normalizadas a norma L2 = 1).
                Por defecto es True.

            force_recompute (bool, optional):
                Si es True, recalcula la matriz ignorando cualquier resultado
                almacenado en caché.
                Por defecto es False.

        Returns:
            numpy.ndarray:
                Matriz de tamaño (N, V) que representa cada documento como
                un vector TF-IDF.
                Si `normalize=True`, las filas estarán normalizadas.
        """

        if (not force_recompute) and hasattr(self, "_doc_matrix_norm") and normalize:
            return self._doc_matrix_norm
        if (not force_recompute) and hasattr(self, "_doc_matrix") and not normalize:
            return self._doc_matrix

        N = len(self.docs)
        V = len(self.word_list)
        M = np.zeros((N, V), dtype=np.float64)

        for doc_id in range(N):
            M[doc_id, :] = self.create_tf_idf(doc_id, is_document=True)

        self._doc_matrix = M

        if normalize:
            norms = np.linalg.norm(M, axis=1, keepdims=True)
            norms[norms == 0.0] = 1.0
            self._doc_matrix_norm = M / norms
            return self._doc_matrix_norm
        else:
            return self._doc_matrix

    def cosine_search(self, query_vector, use_cached=True):
        """
        Calcula las similitudes coseno entre un vector de consulta y todos los
        documentos del índice, devolviendo los resultados ordenados de mayor a menor.

        Este método normaliza el vector de consulta y lo multiplica por la matriz
        de documentos ya normalizada (`_doc_matrix_norm`). Si dicha matriz no existe,
        se construye llamando internamente a `_build_document_matrix(normalize=True, force_recompute=True)`.

        Args:
            query_vector (array-like): Vector de consulta de forma (d,) o (1, d).
                Debe tener la misma dimensionalidad que las columnas de la matriz de
                documentos interna.
            use_cached (bool, opcional): Indicador para usar resultados en caché si
                están disponibles. Actualmente no se utiliza en la implementación,
                pero se mantiene por compatibilidad. Por defecto `True`.

        Returns:
            list[tuple[int, float]]: Lista de tuplas `(doc_id, similitud)` ordenada
            de mayor a menor similitud coseno. Si el vector de consulta es el vector
            cero, devuelve una lista vacía.

        Notas:
            - Complejidad temporal aproximada: O(N·d) para el producto matriz–vector,
            más O(N log N) por la ordenación de todas las similitudes.
            - Requiere que `self._doc_matrix_norm` exista o pueda construirse; esta
            matriz debe estar fila-normalizada (norma L2 = 1 por documento).
            - La similitud coseno resultante está en el rango [-1, 1], aunque en
            contextos de embeddings usualmente se concentra en [0, 1].

        Raises:
            ValueError: Puede propagarse desde NumPy si la dimensionalidad del
            `query_vector` no coincide con la de la matriz de documentos.
            TypeError: Puede propagarse si `query_vector` no es convertible a `np.float64`.

        Ejemplo:
            >>> idx = MyIndex(...)
            >>> vec = encoder("una consulta")
            >>> resultados = idx.cosine_search(vec)
            >>> # primeros 5 documentos más similares
            >>> resultados[:5]
            [(12, 0.8421), (7, 0.8283), (3, 0.8179), (19, 0.8054), (5, 0.7992)]
        """
        if hasattr(self, "_doc_matrix_norm") and self._doc_matrix_norm is not None:
            M = self._doc_matrix_norm
        else:
            self._build_document_matrix(normalize=True, force_recompute=True)
            M = self._doc_matrix_norm

        q = np.asarray(query_vector, dtype=np.float64).ravel()
        nq = np.linalg.norm(q)
        if nq == 0.0:
            return []

        q /= nq
        sims = M.dot(q)

        # Ordenar todas las similitudes de mayor a menor
        order = np.argsort(-sims)

        return [(int(i), float(sims[i])) for i in order]

### [10p] Cree su propia implementación del índice invertido usando los 331 documentos en el conjunto de datos.

Opcion 1:


In [110]:
inverted_index_f(list_docs_files)

({'1': ['1',
   222,
   [[1, 1],
    [2, 1],
    [3, 2],
    [5, 3],
    [6, 1],
    [8, 3],
    [9, 4],
    [12, 3],
    [13, 2],
    [14, 2],
    [16, 1],
    [17, 2],
    [18, 2],
    [19, 1],
    [20, 5],
    [21, 2],
    [22, 3],
    [23, 2],
    [24, 1],
    [25, 1],
    [29, 2],
    [34, 1],
    [35, 2],
    [36, 1],
    [37, 2],
    [38, 1],
    [39, 4],
    [40, 4],
    [42, 2],
    [43, 2],
    [44, 1],
    [45, 1],
    [46, 1],
    [47, 1],
    [48, 2],
    [49, 1],
    [50, 3],
    [51, 1],
    [52, 2],
    [53, 1],
    [54, 2],
    [55, 1],
    [56, 4],
    [58, 1],
    [59, 3],
    [60, 4],
    [61, 1],
    [62, 4],
    [63, 4],
    [65, 2],
    [66, 3],
    [67, 2],
    [68, 3],
    [70, 3],
    [71, 3],
    [72, 3],
    [73, 2],
    [74, 3],
    [75, 2],
    [76, 2],
    [77, 2],
    [78, 2],
    [79, 2],
    [80, 4],
    [81, 4],
    [82, 1],
    [83, 2],
    [84, 1],
    [85, 3],
    [86, 4],
    [87, 1],
    [88, 3],
    [89, 3],
    [90, 4],
    [91, 2],
    [92, 1]

Opcion 2:

In [111]:
list_docs_files = sorted(list_docs_files)
inverted_index = InvertedIndex(30000)

start_time = time.time()

for document in list_docs_files:

    text = get_content_and_title(document)
    tokens = process_text_stop_words(text)
    index = int(document.split(".")[1].strip("d"))
    inverted_index.docs.append(index)

    for token in tokens:
        # print(index)
        inverted_index.insert(token, index)

end_time = time.time()  # tiempo final

print(f"Tiempo total: {end_time - start_time:.4f} segundos")

Tiempo total: 5.8776 segundos


### [10p] Cree una función que lea el índice invertido y calcule consultas booleanas mediante el algoritmo de mezcla. El algoritmo de mezcla debe ser capaz de calcular: AND, y NOT

Para utilizar la función requerida, se debe llamar a free_text_query(). Esta puede recibir un string con el formato "Palabra1 Operador1 Palabra2 Operador2 ... OperadorN PalabraN" o una lista de palabras simple, por ejemplo "HELLO WORLD". Si se utiliza la primera opción, se debe establecer personalized_list=True. En cambio, si se desea que las palabras se unan automáticamente con AND —es decir, que se busque la intersección entre todas ellas—, basta con pasar un string como parámetro sin activar personalized_list.

Ejemplos:


In [112]:
text = f"{STEMMER_EN.stem('English')} {STEMMER_EN.stem('physician')}"
inverted_index.free_text_query(text)

[12, 15, 35, 49, 54, 79, 120, 241, 275, 279, 309, 310]

In [113]:
text

'english physician'

In [114]:
text = "english AND physician"
inverted_index.free_text_query(text, personalized_list=True)

[12, 15, 35, 49, 54, 79, 120, 241, 275, 279, 309, 310]

In [115]:
text = "english NOT physician"
inverted_index.free_text_query(text, personalized_list=True)

[60,
 72,
 77,
 80,
 84,
 85,
 98,
 102,
 109,
 125,
 129,
 132,
 136,
 141,
 143,
 144,
 147,
 152,
 159,
 161,
 165,
 168,
 174,
 180,
 183,
 185,
 190,
 191,
 193,
 194,
 211,
 233,
 237,
 240,
 243,
 246,
 248,
 259,
 260,
 263,
 264,
 267,
 273,
 289,
 292,
 295,
 297,
 298,
 299,
 304,
 306,
 316]

In [116]:
text = "english NOT physician AND physician"
inverted_index.free_text_query(text, personalized_list=True)

[]

### [5p] Para cada una de las 35 consultas en el conjunto de datos, recupere los documentos utilizando consultas binarias AND (i.e. termino_1 AND termino_2 AND termino_3…). Escriba un archivo (BSII-AND-queries_results) con los resultados siguiendo el mismo formato que "relevance-judgments":

In [ ]:
list_query_files = sorted(
    os.listdir("./data/queries-raw-texts/")
)  # tiene que existir la carpeta de data
output_path = "BSII-AND-queries_results"

start_time = time.time()

with open(output_path, "w", encoding="utf-8") as out:
    for document in list_query_files:
        text = get_content(document)

        tokens = process_text_stop_words(text)
        second_text = ""
        for word in tokens:
            second_text += word + " "
        second_text = second_text.strip()

        if len(tokens) == 1:
            inverted_index.get(tokens[0])
        else:
            doc_ids = inverted_index.free_text_query(second_text) or []
        qnum = int(document.split(".")[1].strip("q"))
        qid = f"q{qnum:02d}"

        docs_str = ",".join(f"d{int(d):03d}" for d in doc_ids)
        line = f"{qid} {docs_str}" if docs_str else f"{qid} "
        out.write(line + "\n")

end_time = time.time()
print(f"Listo: escrito '{output_path}' en {end_time - start_time:.3f}s")

Listo: escrito 'BSII-AND-queries_results' en 0.010s


## [35p] Recuperación ranqueada y vectorización de documentos (RRDV)

### [10p] Cree una función que, a partir del índice invertido, cree la representación vectorial ponderada tf.idf de un documento o consulta. Describa en detalle su estrategia, ¿es eficiente? ¿por qué si, por qué no?

In [ ]:
text = "william beaumont and Queen of France Catherine de’ Medici was born. Catherine played a key role in the reign of her sons, and is blamed for four apach"
tokens = process_text_stop_words(text)

# En caso de querer hacer el vector de un nuevo texto, en este caso
inverted_index.create_tf_idf(token_list=tokens, is_document=False)

array([0.24710288, 0.75854381, 0.        , ..., 0.        , 0.        ,
       0.75854381])

In [119]:
# En caso de querer hacerlo de un documento ya existente se le pasa la id
inverted_index.create_tf_idf(1)

array([0.69370548, 2.88804551, 0.44653389, ..., 0.        , 0.        ,
       0.        ])

La estrategia es eficiente a la hora de crearlos, dado que guarda el tf el df y el N, es facil generar lo vectores, solo es ir por palabra a consultar las palabras del vocabulario guardadas y eso es O(n).
Mientras que para las fases es O(n2), es malo, porque va recorriendo por token toda la lista una y otra vez por cada tipo distinto de token (se arregla pasando una vez y contando el tf de los tokens que es el verdadero problema).


### [10p] Cree una función que reciba dos vectores de documentos y calcule la similitud del coseno

In [ ]:
inverted_index.cosine_similarity([1, 2], [2, 1])

0.7999999999999998

Tiene las mismas restricciones que una similaridad coseno.

### [5p] Para cada una de las 35 consultas en el conjunto de datos, recupere los documentos clasificados - ordenados por el puntaje de similitud del coseno- (incluya solo los documentos con un puntaje superior a 0 para una consulta determinada). Escriba un archivo (RRDV-consultas_resultados) con los resultados siguiendo el siguiente formato: 

### q01 dXX: cos_simi(q01,dXX),dYY: cos_simi(q01, dYY),dZZ: cos_simi(q01,dZZ)…

In [ ]:
list_query_files = sorted(os.listdir("./data/queries-raw-texts/"))
output_path = "RRDV-consultas_resultado"

start_time = time.time()

with open(output_path, "w", encoding="utf-8") as out:
    for document in list_query_files:
        qid = int(document.split(".")[1].strip("q"))
        text = get_content(document)

        tokens = process_text_stop_words(text)
        vector_tf_idf = inverted_index.create_tf_idf(
            1, token_list=tokens, is_document=False
        )

        results = inverted_index.cosine_search(vector_tf_idf)

        results = [(doc, score) for doc, score in results if score > 0]
        results.sort(key=lambda x: x[1], reverse=True)

        docs_str = ",".join(f"d{int(doc):03d}:{score:.4f}" for doc, score in results)

        line = f"q{qid:02d} {docs_str}" if docs_str else f"q{qid:02d} "

        out.write(line + "\n")

end_time = time.time()
print(f"Listo: escrito '{output_path}' en {end_time - start_time:.3f}s")

Listo: escrito 'RRDV-consultas_resultado' en 14.615s


### [10p] Evaluación de resultados. Calcule P@M, R@M, NDCG@M por consulta. M es el número de documentos relevantes encontrados en el archivo de juicios de relevancia por consulta. Luego calcule MAP como una métrica general.

NOTA I: Para P@M y R@M suponga una escala de relevancia binaria. Los documentos que no se
encuentran en el archivo “relevance-judgments” NO son relevantes para una consulta determinada

In [122]:
q_to_docs = {}
# Para poder sacar las metricas de P y R, necesitamos saver cuales estan bien.
with open("./data/relevance-judgments.tsv", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        qid, pairs = line.split(None, 1)

        m = re.search(r"(\d+)", qid)
        if not m:
            continue
        qnum = int(m.group(1))
        docs = []
        for p in pairs.split(","):
            p = p.strip()
            if not p:
                continue
            doc_id = int(p.split(":", 1)[0].replace("d", ""))
            docs.append(doc_id)

        q_to_docs[qnum] = docs
q_to_docs

{1: [186, 254, 16],
 2: [136, 139, 143, 283, 228, 164, 318, 291, 293, 147, 149],
 3: [152, 291, 283, 147, 318, 105],
 4: [275, 10, 286, 19, 49, 330, 270],
 6: [69, 233, 257, 297, 26, 329],
 7: [4, 77, 266, 179],
 8: [205, 5, 110, 108, 117, 81, 292, 251, 28, 271, 121, 180],
 9: [205, 199, 198, 223, 217, 177],
 10: [68, 100, 65, 76, 231, 199, 52, 215],
 12: [239, 277, 258, 250],
 13: [239, 277, 258, 49, 56],
 14: [2, 5, 142, 314, 280, 130, 41, 117, 81, 93, 91, 180],
 16: [229, 132],
 17: [280, 271, 121, 91],
 18: [207, 201, 192, 194, 222, 216, 210],
 19: [77, 179],
 22: [277, 11, 132, 258, 49, 250, 331],
 23: [205, 202, 276, 194, 216, 219, 215, 211],
 24: [98, 129, 196, 221, 60],
 25: [167, 166, 20, 23],
 26: [152],
 27: [103, 143, 107, 51, 17, 54, 293, 158],
 28: [136, 316, 94],
 29: [1, 37, 130, 314, 46, 133, 113, 294, 261, 93, 62, 120],
 32: [139, 67, 25, 31, 90],
 34: [248],
 36: [277, 167, 257, 20, 23, 321, 247, 265, 150, 328],
 37: [169, 256, 116],
 38: [235, 39, 229, 317, 15, 250,

In [ ]:
query_results_matrix = []
relevant_counts = []

for document in list_query_files:
    qid = int(document.split(".")[1].strip("q"))
    text = get_content(document)
    tokens = process_text_stop_words(text)

    vector_tf_idf = inverted_index.create_tf_idf(
        1, token_list=tokens, is_document=False
    )

    results = inverted_index.cosine_search(vector_tf_idf)
    # print(results)
    results = [doc for doc, score in results if score > 0]

    if qid in q_to_docs:
        ground_truth = q_to_docs[qid]
        m = len(ground_truth)  # número de documentos relevantes
        final_result = [1 if doc in ground_truth else 0 for doc in results]

        final_result = final_result[:m]

        relevant_counts.append(m)
        query_results_matrix.append(np.array(final_result))
    # else:
    # final_result = [0 for doc in results]
    # relevant_counts.append(0)


map_score = mean_average_precision(query_results_matrix)
mar_score = mean_average_recall(query_results_matrix, relevant_counts)

print("MAP:", map_score)
print("MAR:", mar_score)

MAP: 0.8693144961928976
MAR: 0.4426113172541744


NOTA II: Para NDCG@M utilice la escala de relevancia no binaria que se encuentra en el archivo
“relevance-judgments”

In [124]:
q_to_docs = {}

with open("./data/relevance-judgments.tsv", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        qid, pairs = line.split(None, 1)
        m = re.search(r"(\d+)", qid)
        if not m:
            continue
        qnum = int(m.group(1))

        docs_dict = {}
        for p in pairs.split(","):
            p = p.strip()
            if not p:
                continue
            doc_id_str, value_str = p.split(":")
            doc_id = int(doc_id_str.replace("d", ""))
            value = int(value_str)
            docs_dict[doc_id] = value

        q_to_docs[qnum] = docs_dict
q_to_docs

{1: {186: 4, 254: 5, 16: 5},
 2: {136: 2,
  139: 2,
  143: 4,
  283: 4,
  228: 4,
  164: 4,
  318: 2,
  291: 4,
  293: 4,
  147: 2,
  149: 2},
 3: {152: 3, 291: 4, 283: 4, 147: 3, 318: 2, 105: 2},
 4: {275: 3, 10: 3, 286: 2, 19: 2, 49: 2, 330: 2, 270: 3},
 6: {69: 2, 233: 3, 257: 2, 297: 3, 26: 4, 329: 5},
 7: {4: 3, 77: 3, 266: 2, 179: 3},
 8: {205: 2,
  5: 4,
  110: 4,
  108: 3,
  117: 3,
  81: 2,
  292: 2,
  251: 5,
  28: 3,
  271: 3,
  121: 2,
  180: 2},
 9: {205: 3, 199: 5, 198: 3, 223: 2, 217: 2, 177: 2},
 10: {68: 2, 100: 2, 65: 3, 76: 3, 231: 4, 199: 4, 52: 2, 215: 2},
 12: {239: 4, 277: 4, 258: 3, 250: 4},
 13: {239: 2, 277: 2, 258: 2, 49: 4, 56: 4},
 14: {2: 2,
  5: 3,
  142: 2,
  314: 3,
  280: 3,
  130: 3,
  41: 3,
  117: 2,
  81: 4,
  93: 3,
  91: 4,
  180: 3},
 16: {229: 2, 132: 3},
 17: {280: 2, 271: 4, 121: 4, 91: 2},
 18: {207: 2, 201: 3, 192: 4, 194: 3, 222: 2, 216: 2, 210: 3},
 19: {77: 2, 179: 5},
 22: {277: 2, 11: 3, 132: 2, 258: 2, 49: 2, 250: 2, 331: 2},
 23: {20

In [ ]:
query_results_matrix = []
relevant_counts = []
ndcg_scores = []
for document in list_query_files:
    qid = int(document.split(".")[1].strip("q"))
    text = get_content(document)
    tokens = process_text_stop_words(text)

    vector_tf_idf = inverted_index.create_tf_idf(
        1, token_list=tokens, is_document=False
    )

    results = inverted_index.cosine_search(vector_tf_idf)

    results = [doc for doc, score in results if score > 0]

    if qid in q_to_docs:
        ground_truth = q_to_docs[qid]
        m = len(ground_truth)
    
        final_result = [ground_truth.get(doc, 0) for doc in results]

        final_result = final_result[:m]

        relevant_counts.append(m)
        query_results_matrix.append(np.array(final_result))
        ndcg_scores.append(ndcg_at_k(final_result, m))

average_ndcg = np.mean(ndcg_scores)
print(f"NDCG promedio: {average_ndcg:.4f}")

NDCG promedio: 0.9058


### [30] Repita el punto anterior (RRDV) pero utilizando GENSIM (use como nombre de archivo GESIM-consultas_resultados)

In [ ]:
q_to_docs = {}

with open("./data/relevance-judgments.tsv", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        qid, pairs = line.split(None, 1)

        m = re.search(r"(\d+)", qid)
        if not m:
            continue
        qnum = int(m.group(1))
        docs = []
        for p in pairs.split(","):
            p = p.strip()
            if not p:
                continue
            doc_id = int(p.split(":", 1)[0].replace("d", ""))
            docs.append(doc_id)

        q_to_docs[qnum] = docs
q_to_docs

{1: [186, 254, 16],
 2: [136, 139, 143, 283, 228, 164, 318, 291, 293, 147, 149],
 3: [152, 291, 283, 147, 318, 105],
 4: [275, 10, 286, 19, 49, 330, 270],
 6: [69, 233, 257, 297, 26, 329],
 7: [4, 77, 266, 179],
 8: [205, 5, 110, 108, 117, 81, 292, 251, 28, 271, 121, 180],
 9: [205, 199, 198, 223, 217, 177],
 10: [68, 100, 65, 76, 231, 199, 52, 215],
 12: [239, 277, 258, 250],
 13: [239, 277, 258, 49, 56],
 14: [2, 5, 142, 314, 280, 130, 41, 117, 81, 93, 91, 180],
 16: [229, 132],
 17: [280, 271, 121, 91],
 18: [207, 201, 192, 194, 222, 216, 210],
 19: [77, 179],
 22: [277, 11, 132, 258, 49, 250, 331],
 23: [205, 202, 276, 194, 216, 219, 215, 211],
 24: [98, 129, 196, 221, 60],
 25: [167, 166, 20, 23],
 26: [152],
 27: [103, 143, 107, 51, 17, 54, 293, 158],
 28: [136, 316, 94],
 29: [1, 37, 130, 314, 46, 133, 113, 294, 261, 93, 62, 120],
 32: [139, 67, 25, 31, 90],
 34: [248],
 36: [277, 167, 257, 20, 23, 321, 247, 265, 150, 328],
 37: [169, 256, 116],
 38: [235, 39, 229, 317, 15, 250,

In [ ]:
list_query_files = sorted(os.listdir("./data/queries-raw-texts/"))


def get_content_gensim(document, is_document=True):
    if is_document:
        tree = ET.parse(f"./data/docs-raw-texts/{document}")
        root = tree.getroot()
        file_desc = root.find(".//fileDesc")
        element_raw = root.find("raw")
        if file_desc is not None:
            title = file_desc.attrib.get("title")
        text = f"{title}\n {element_raw.text}"
        idx = int(document.split(".")[1].strip("d"))
    else:
        tree = ET.parse(f"./data/queries-raw-texts/{document}")
        root = tree.getroot()
        element_raw = root.find("raw").text
        text = element_raw
        idx = int(document.split(".")[1].strip("q"))
    return idx, text


lists_tokens_docs = []
docs_ids = []
for document in list_docs_files:
    idx, text = get_content_gensim(document)
    text = simple_preprocess(text, deacc=True, min_len=2, max_len=30)
    lists_tokens_docs.append(text)
    docs_ids.append(idx)

# Obtengo bow
dictionary = Dictionary(lists_tokens_docs)
bow_corpus = [dictionary.doc2bow(tokens) for tokens in lists_tokens_docs]
# Transformo en TF IDF
tfidf = models.TfidfModel(bow_corpus, normalize=True)
tfidf_corpus = tfidf[bow_corpus]
# Indice basado en similitud coseno
index = similarities.SparseMatrixSimilarity(tfidf_corpus, num_features=len(dictionary))
# Desarrollo de los resultados de las queries

#### P@M y R@M 

In [ ]:
query_results_matrix = []
relevant_counts = []

salida = []

for document in list_query_files:
    idx, text = get_content_gensim(document, is_document=False)
    text = simple_preprocess(text, deacc=True, min_len=2, max_len=30)
    query_vector = tfidf[dictionary.doc2bow(text)]

    sims = index[query_vector]

    pares = [(docs_ids[i], float(sims[i])) for i in range(len(docs_ids))]
    pares = [(d, s) for (d, s) in pares if s > 0.0]

    pares.sort(key=lambda x: x[1], reverse=True)

    if pares:
        linea = "q" + str(idx) + " " + ", ".join([f"{d}: {s:.6f}" for d, s in pares])
    else:
        linea = idx
    salida.append((idx, linea))

    if idx in q_to_docs:
        ground_truth = q_to_docs[idx]
        final_result = [1 if doc[0] in ground_truth else 0 for doc in pares]

        final_result_ndcg = []
        relevant_counts.append(len(ground_truth))
    # else:
    # final_result = [0 for doc in pares]
    # relevant_counts.append(0)
    query_results_matrix.append(np.array(final_result))

with open("GESIM-consultas_resultados", "w", encoding="utf-8") as f:
    for _, linea in salida:
        f.write(linea + "\n")

print("Archivo escrito: GESIM-consultas_resultados")

map_score = mean_average_precision(query_results_matrix)
mar_score = mean_average_recall(query_results_matrix, relevant_counts)
ndcg = ndcg_at_k
print("MAP:", map_score)
print("MAR:", mar_score)

Archivo escrito: GESIM-consultas_resultados
MAP: 0.6879069408798658
MAR: 0.6035492681921253


### NDCG@M 

In [129]:
q_to_docs = {}

with open("./data/relevance-judgments.tsv", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        qid, pairs = line.split(None, 1)
        m = re.search(r"(\d+)", qid)
        if not m:
            continue
        qnum = int(m.group(1))

        docs_dict = {}
        for p in pairs.split(","):
            p = p.strip()
            if not p:
                continue
            doc_id_str, value_str = p.split(":")
            doc_id = int(doc_id_str.replace("d", ""))
            value = int(value_str)
            docs_dict[doc_id] = value

        q_to_docs[qnum] = docs_dict
q_to_docs

{1: {186: 4, 254: 5, 16: 5},
 2: {136: 2,
  139: 2,
  143: 4,
  283: 4,
  228: 4,
  164: 4,
  318: 2,
  291: 4,
  293: 4,
  147: 2,
  149: 2},
 3: {152: 3, 291: 4, 283: 4, 147: 3, 318: 2, 105: 2},
 4: {275: 3, 10: 3, 286: 2, 19: 2, 49: 2, 330: 2, 270: 3},
 6: {69: 2, 233: 3, 257: 2, 297: 3, 26: 4, 329: 5},
 7: {4: 3, 77: 3, 266: 2, 179: 3},
 8: {205: 2,
  5: 4,
  110: 4,
  108: 3,
  117: 3,
  81: 2,
  292: 2,
  251: 5,
  28: 3,
  271: 3,
  121: 2,
  180: 2},
 9: {205: 3, 199: 5, 198: 3, 223: 2, 217: 2, 177: 2},
 10: {68: 2, 100: 2, 65: 3, 76: 3, 231: 4, 199: 4, 52: 2, 215: 2},
 12: {239: 4, 277: 4, 258: 3, 250: 4},
 13: {239: 2, 277: 2, 258: 2, 49: 4, 56: 4},
 14: {2: 2,
  5: 3,
  142: 2,
  314: 3,
  280: 3,
  130: 3,
  41: 3,
  117: 2,
  81: 4,
  93: 3,
  91: 4,
  180: 3},
 16: {229: 2, 132: 3},
 17: {280: 2, 271: 4, 121: 4, 91: 2},
 18: {207: 2, 201: 3, 192: 4, 194: 3, 222: 2, 216: 2, 210: 3},
 19: {77: 2, 179: 5},
 22: {277: 2, 11: 3, 132: 2, 258: 2, 49: 2, 250: 2, 331: 2},
 23: {20

In [ ]:
query_results_matrix = []
relevant_counts = []
ndcg_scores = []
salida = []

for document in list_query_files:
    idx, text = get_content_gensim(document, is_document=False)
    text = simple_preprocess(text, deacc=True, min_len=2, max_len=30)
    query_vector = tfidf[dictionary.doc2bow(text)]

    sims = index[query_vector]

    pares = [(docs_ids[i], float(sims[i])) for i in range(len(docs_ids))]
    pares = [(d, s) for (d, s) in pares if s > 0.0]
    pares.sort(key=lambda x: x[1], reverse=True)

    if pares:
        linea = "q" + str(idx) + " " + ", ".join([f"{d}: {s:.6f}" for d, s in pares])
    else:
        linea = "q" + str(idx)
    salida.append((idx, linea))

    if idx in q_to_docs:
        ground_truth = q_to_docs[idx]
        relevant_counts.append(len(ground_truth))

        final_result = [ground_truth.get(doc[0], 0) for doc in pares]

        ndcg_scores.append(ndcg_at_k(np.array(final_result), k=len(final_result)))

        query_results_matrix.append(np.array(final_result))
    else:
        query_results_matrix.append(np.array([]))
        ndcg_scores.append(0.0)
        relevant_counts.append(0)

average_ndcg = np.mean(ndcg_scores)
print(f"NDCG promedio: {average_ndcg:.4f}")

NDCG promedio: 0.8003
